In [1]:
BROKERS = ["host.docker.internal:9092"]

In [3]:
#from kafka import KafkaProducer, KafkaConsumer
import json
import threading
import time
import uuid
from pcomp_utils.kafka_producer_utils import KafkaProducerHandler
from pcomp_utils.kafka_consumer_utils import KafkaConsumerHandler

KAFKA_BROKER = 'kafka:9092'

def produce_messages():
    # producer = KafkaProducer(
    #     bootstrap_servers=BROKERS,
    #     value_serializer=lambda v: json.dumps(v).encode('utf-8')
    # )
    producer = KafkaProducerHandler(KAFKA_BROKER)

    for i in range(5):
        message = {
            "user_id": i,
            "action": "activate",
            "timestamp": "2025-03-12T12:00:00"
        }
        producer.send('activate-layer', message, 0)
        print("Message sent: ", message)
        time.sleep(1)

    producer.close()

def consume_messages():
    consumer = KafkaConsumerHandler('activate-layer', KAFKA_BROKER, partition=0)

    print("Starting consumer...")
    count = 0
    for message in consumer.consume():
        time.sleep(2)
        print("Received message: ", message.value)
        count += 1
        if count >= 5:
            break

if __name__ == "__main__":
    # Start consumer in a separate thread
    consumer_thread = threading.Thread(target=consume_messages)
    consumer_thread.start()

    # Ensure the consumer is fully ready
    time.sleep(2)

    produce_messages()

    consumer_thread.join()


Starting consumer...
Message sent:  {'user_id': 0, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Message sent:  {'user_id': 1, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message:  {'user_id': 0, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Message sent:  {'user_id': 2, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Message sent:  {'user_id': 3, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message:  {'user_id': 1, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Message sent:  {'user_id': 4, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message:  {'user_id': 2, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message:  {'user_id': 3, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
Received message:  {'user_id': 4, 'action': 'activate', 'timestamp': '2025-03-12T12:00:00'}
